In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
import torch
# Disable torch compile for Windows compatibility
import torch._dynamo
torch._dynamo.config.disable = True

from datetime import datetime, timezone
import logging
import copy

logging.basicConfig(
    level=logging.INFO,  # Set the logging level
    format='%(asctime)s - %(levelname)s - %(message)s',  # Format for the log messages
    handlers=[
        logging.StreamHandler()  # Log to the console
    ]
)

%reload_ext autoreload
%autoreload 2
from core_data_prep.core_data_prep import DataPreparer
from core_data_prep.validations import Validator

from data.raw.retrievers.alpaca_markets_retriever import AlpacaMarketsRetriever
from data.raw.retrievers.stooq_retriever import StooqRetriever
from config.constants import *
from data.processed.dataset_creation import DatasetCreator
from data.processed.indicators import *
from data.processed.targets import Balanced3ClassClassification
from data.processed.normalization import ZScoreOverWindowNormalizer, ZScoreNormalizer, MinMaxNormalizer
from data.processed.dataset_pytorch import DatasetPytorch
from modeling.trainer import Trainer
from modeling.evaluate import evaluate_lgb_regressor, evaluate_torch_regressor, evaluate_torch_regressor_multiasset
from modeling.modeling_utils import print_model_parameters

from modeling.rl.environment import PortfolioEnvironment
from modeling.rl.state import State
from modeling.rl.agent import RlAgent
from modeling.rl.algorithms.policy_gradient import PolicyGradient
from modeling.rl.actors.actor import RlActor
from modeling.rl.actors.signal_predictor_actor import SignalPredictorActor
from modeling.rl.actors.high_energy_low_friction_actor import HighEnergyLowFrictionActor
from modeling.rl.actors.xsmom_actor import XSMomActor
from modeling.rl.actors.tsmom_actor import TSMomActor
from modeling.rl.actors.blsw_actor import BLSWActor
from modeling.rl.actors.allocation_propogation_actor import AllocationPropogationActor
from modeling.rl.actors.market_actor import MarketActor
from modeling.rl.trajectory_dataset import TrajectoryDataset
from modeling.rl.metrics import MetricsCalculator, DEFAULT_METRICS
from modeling.rl.reward import EstimatedReturnReward
from modeling.rl.loss import SumLogReturnLoss, ReinforceLoss
from modeling.rl.visualization.wealth_plot import plot_cumulative_wealth
from modeling.rl.visualization.position_plot import plot_position_heatmap
from config.experiments.cur_experiment import config

torch.backends.cudnn.benchmark = config.train_config.cudnn_benchmark

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [2]:
str(config.data_config.frequency)

'1Day'

In [3]:
retriever = config.data_config.retriever
initial_state = config.model_config.model.state_dict()

In [4]:
def evaluate_signal_predictor(symbols: list[str], end_date: datetime, train_set_last_date: datetime, val_set_last_date: datetime, model=None):
    print(f'Running experiment for {config.data_config.start} to {end_date} with train set last date {train_set_last_date} and val set last date {val_set_last_date}')
    
    retrieval_result = retriever.bars_with_quotes(symbol_or_symbols=symbols, start=config.data_config.start, end=end_date)

    data_preparer = DataPreparer(
        normalizer=config.data_config.normalizer,
        missing_values_handler=config.data_config.missing_values_handler_polars,
        in_seq_len=config.data_config.in_seq_len,
        frequency=str(config.data_config.frequency),
        validator=config.data_config.validator,
    )

    (X_train, y_train, statistics_train), (X_val, y_val, statistics_val), (X_test, y_test, statistics_test) = \
        data_preparer.get_experiment_data(
            data=retrieval_result,
            start_date=None,
            end_date=None,
            features=config.data_config.features_polars,
            statistics=config.data_config.statistics,
            target=config.data_config.target,
            train_set_last_date=train_set_last_date,
            val_set_last_date=val_set_last_date,
        )
    next_return_train, spread_train, volatility_train, \
        next_return_val, spread_val, volatility_val, \
        next_return_test, spread_test, volatility_test = \
            statistics_train['next_return'], statistics_train['spread'], statistics_train['volatility'], \
            statistics_val['next_return'], statistics_val['spread'], statistics_val['volatility'], \
            statistics_test['next_return'], statistics_test['spread'], statistics_test['volatility']
        
    print(X_train.shape, y_train.shape, next_return_train.shape, spread_train.shape, volatility_train.shape, X_test.shape,\
         y_test.shape, next_return_test.shape, spread_test.shape, volatility_test.shape)

    if not model: 
        print('Training model...')
        train_loader = DatasetPytorch(X_train, y_train, learning_task='regression').as_dataloader(
            batch_size=config.train_config.batch_size,
            shuffle=config.train_config.shuffle,
            num_workers=config.train_config.num_workers,
            prefetch_factor=config.train_config.prefetch_factor,
            pin_memory=config.train_config.pin_memory,
            persistent_workers=config.train_config.persistent_workers,
            drop_last=config.train_config.drop_last
        )
        val_loader = DatasetPytorch(X_val, y_val, learning_task='regression').as_dataloader(
            batch_size=config.train_config.batch_size,
            shuffle=config.train_config.shuffle,
            num_workers=config.train_config.num_workers,
            prefetch_factor=config.train_config.prefetch_factor,
            pin_memory=config.train_config.pin_memory,
            persistent_workers=config.train_config.persistent_workers,
            drop_last=config.train_config.drop_last
        )

        model = config.model_config.model
        model.load_state_dict(initial_state)

        trainer = Trainer(
            model=model,
            train_loader=train_loader,
            val_loader=val_loader,
            loss_fn=config.train_config.loss_fn,
            optimizer=config.train_config.optimizer,
            scheduler=config.train_config.scheduler,
            num_epochs=config.train_config.num_epochs,
            early_stopping_patience=config.train_config.early_stopping_patience,
            device=config.train_config.device,
            metrics=config.train_config.metrics,
            save_path=config.train_config.save_path
        )

        model, history = trainer.train()
    else: 
        print('Using existing model...')

    test_trajectory_loader = TrajectoryDataset(X_test, next_return_test, spread_test, volatility_test, \
            trajectory_length=config.rl_config.trajectory_length, horizon=config.data_config.horizon).as_dataloader(
        batch_size=1, 
        shuffle=False,
        num_workers=8,
        prefetch_factor=4,
        pin_memory=True,
        persistent_workers=True,
        drop_last=False,
    )

    env = PortfolioEnvironment(
        reward_function=EstimatedReturnReward(fee=config.rl_config.fee, spread_multiplier=config.rl_config.spread_multiplier),
    )

    actor = RlActor(
        copy.deepcopy(model), 
        n_assets=len(config.data_config.symbol_or_symbols),
        train_signal_predictor=False, 
    ).to(device)

    # signal_predictor_actor = SignalPredictorActor(
    #     copy.deepcopy(model), 
    #     trade_asset_count=config.rl_config.trade_asset_count,
    #     train_signal_predictor=False
    # ).to(device)

    eval_actor = AllocationPropogationActor(
        copy.deepcopy(model), 
        train_allocator=False
    ).to(device)

    # eval_actor = MarketActor().to(device)

    rl_agent = RlAgent(
        actor, 
        env
    )

    metrics_calculator = MetricsCalculator(
        metrics=DEFAULT_METRICS
    )

    policy_gradient = PolicyGradient(
        rl_agent, 
        None, 
        test_trajectory_loader, 
        metrics_calculator=metrics_calculator,
        optimizer=torch.optim.AdamW(
            [p for p in actor.parameters() if p.requires_grad], 
            lr=1e-3,
            weight_decay=1e-5,
            amsgrad=True),
        scheduler=None,
        loss_fn=ReinforceLoss(use_baseline=False),
        num_epochs=10,
        device=device
    )

    epoch_loss, realized_returns_signal_predictor, actions_signal_predictor = policy_gradient.evaluate(eval_actor, test_trajectory_loader)
    return actions_signal_predictor, realized_returns_signal_predictor, model, min(history['val_loss'])


In [5]:
# start_end_last_train_dates = [
#     {'end': datetime(2011, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2010, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2008, 1, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2012, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2011, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2009, 1, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2013, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2012, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2010, 1, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2014, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2013, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2011, 1, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2015, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2014, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2012, 1, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2016, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2015, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2013, 1, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2017, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2016, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2014, 1, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2018, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2017, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2015, 1, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2019, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2018, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2016, 1, 1, tzinfo=timezone.utc)},
# ]
# start_end_last_train_dates = [
#     {'end': datetime(2010, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2009, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2008, 1, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2011, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2010, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2009, 1, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2012, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2011, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2010, 1, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2013, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2012, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2011, 1, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2014, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2013, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2012, 1, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2015, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2014, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2013, 1, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2016, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2015, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2014, 1, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2017, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2016, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2015, 1, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2018, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2017, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2016, 1, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2019, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2018, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2017, 1, 1, tzinfo=timezone.utc)},
# ]
# start_end_last_train_dates = [
#     {'end': datetime(2001, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2000, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(1999, 1, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2002, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2001, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2000, 1, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2004, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2002, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2000, 1, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2006, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2004, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2002, 1, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2008, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2006, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2004, 1, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2010, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2008, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2006, 1, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2012, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2010, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2008, 1, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2014, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2012, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2010, 1, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2019, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2014, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2012, 1, 1, tzinfo=timezone.utc)},
# ]
# start_end_last_train_dates = [
#     {'end': datetime(2000, 7, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2000, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(1999, 7, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2001, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2000, 7, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2000, 1, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2001, 7, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2001, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2000, 7, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2002, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2001, 7, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2001, 1, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2002, 7, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2002, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2001, 7, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2003, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2002, 7, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2002, 1, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2004, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2003, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2002, 1, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2008, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2004, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2002, 1, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2014, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2008, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2006, 1, 1, tzinfo=timezone.utc)},
#     {'end': datetime(2019, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2014, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2012, 1, 1, tzinfo=timezone.utc)},
# ]

# start_end_last_train_dates = [
#     {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2004, 4, 8, tzinfo=timezone.utc), 'last_val_date': datetime(2000, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(1999, 1, 1, tzinfo=timezone.utc)},
#     {'symbols': Constants.Data.DJIA_2004_2008, 'end': datetime(2008, 2, 19, tzinfo=timezone.utc), 'last_val_date': datetime(2004, 4, 8, tzinfo=timezone.utc), 'last_train_date': datetime(2003, 4, 8, tzinfo=timezone.utc)},
#     {'symbols': Constants.Data.DJIA_2008_2008, 'end': datetime(2008, 9, 22, tzinfo=timezone.utc), 'last_val_date': datetime(2008, 2, 19, tzinfo=timezone.utc), 'last_train_date': datetime(2007, 2, 19, tzinfo=timezone.utc)},
#     {'symbols': Constants.Data.DJIA_2008_2009, 'end': datetime(2009, 6, 8, tzinfo=timezone.utc), 'last_val_date': datetime(2008, 9, 22, tzinfo=timezone.utc), 'last_train_date': datetime(2007, 9, 22, tzinfo=timezone.utc)},
#     {'symbols': Constants.Data.DJIA_2009_2012, 'end': datetime(2012, 9, 24, tzinfo=timezone.utc), 'last_val_date': datetime(2009, 6, 8, tzinfo=timezone.utc), 'last_train_date': datetime(2008, 6, 8, tzinfo=timezone.utc)},
#     {'symbols': Constants.Data.DJIA_2012_2013, 'end': datetime(2013, 9, 23, tzinfo=timezone.utc), 'last_val_date': datetime(2012, 9, 24, tzinfo=timezone.utc), 'last_train_date': datetime(2011, 9, 24, tzinfo=timezone.utc)},
#     {'symbols': Constants.Data.DJIA_2013_2015, 'end': datetime(2015, 3, 19, tzinfo=timezone.utc), 'last_val_date': datetime(2013, 9, 23, tzinfo=timezone.utc), 'last_train_date': datetime(2012, 9, 23, tzinfo=timezone.utc)},
#     {'symbols': Constants.Data.DJIA_2015_2017, 'end': datetime(2017, 3, 19, tzinfo=timezone.utc), 'last_val_date': datetime(2015, 3, 19, tzinfo=timezone.utc), 'last_train_date': datetime(2014, 3, 19, tzinfo=timezone.utc)},
#     {'symbols': Constants.Data.DJIA_2017_2018, 'end': datetime(2018, 6, 26, tzinfo=timezone.utc), 'last_val_date': datetime(2017, 9, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2016, 9, 1, tzinfo=timezone.utc)},
#     {'symbols': Constants.Data.DJIA_2018_2019, 'end': datetime(2019, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2018, 6, 26, tzinfo=timezone.utc), 'last_train_date': datetime(2017, 6, 26, tzinfo=timezone.utc)},
# ]
# start_end_last_train_dates = [
#     # {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2004, 4, 8, tzinfo=timezone.utc), 'last_val_date': datetime(2000, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(1999, 1, 1, tzinfo=timezone.utc)},
#     # {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2008, 2, 19, tzinfo=timezone.utc), 'last_val_date': datetime(2004, 4, 8, tzinfo=timezone.utc), 'last_train_date': datetime(2003, 4, 8, tzinfo=timezone.utc)},
#     # {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2008, 9, 22, tzinfo=timezone.utc), 'last_val_date': datetime(2008, 2, 19, tzinfo=timezone.utc), 'last_train_date': datetime(2007, 2, 19, tzinfo=timezone.utc)},
#     # {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2009, 6, 8, tzinfo=timezone.utc), 'last_val_date': datetime(2008, 9, 22, tzinfo=timezone.utc), 'last_train_date': datetime(2007, 9, 22, tzinfo=timezone.utc)},
#     {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2012, 9, 24, tzinfo=timezone.utc), 'last_val_date': datetime(2009, 6, 8, tzinfo=timezone.utc), 'last_train_date': datetime(2008, 6, 8, tzinfo=timezone.utc)},
#     {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2013, 9, 23, tzinfo=timezone.utc), 'last_val_date': datetime(2012, 9, 24, tzinfo=timezone.utc), 'last_train_date': datetime(2011, 9, 24, tzinfo=timezone.utc)},
#     {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2015, 3, 19, tzinfo=timezone.utc), 'last_val_date': datetime(2013, 9, 23, tzinfo=timezone.utc), 'last_train_date': datetime(2012, 9, 23, tzinfo=timezone.utc)},
#     {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2016, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2015, 3, 19, tzinfo=timezone.utc), 'last_train_date': datetime(2014, 3, 19, tzinfo=timezone.utc)},
#     {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2017, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2016, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2015, 1, 1, tzinfo=timezone.utc)},
#     {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2018, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2017, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2016, 1, 1, tzinfo=timezone.utc)},
#     {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2019, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2018, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2018, 7, 1, tzinfo=timezone.utc)},
# ]
start_end_last_train_dates = [
    # {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2004, 4, 8, tzinfo=timezone.utc), 'last_val_date': datetime(2000, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(1999, 1, 1, tzinfo=timezone.utc)},
    # {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2008, 2, 19, tzinfo=timezone.utc), 'last_val_date': datetime(2004, 4, 8, tzinfo=timezone.utc), 'last_train_date': datetime(2003, 4, 8, tzinfo=timezone.utc)},
    # {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2008, 9, 22, tzinfo=timezone.utc), 'last_val_date': datetime(2008, 2, 19, tzinfo=timezone.utc), 'last_train_date': datetime(2007, 2, 19, tzinfo=timezone.utc)},
    # {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2009, 6, 8, tzinfo=timezone.utc), 'last_val_date': datetime(2008, 9, 22, tzinfo=timezone.utc), 'last_train_date': datetime(2007, 9, 22, tzinfo=timezone.utc)},
    # {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2001, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2000, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(1999, 1, 1, tzinfo=timezone.utc)},
    # {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2002, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2001, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2000, 1, 1, tzinfo=timezone.utc)},
    # {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2003, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2002, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2001, 1, 1, tzinfo=timezone.utc)},
    # {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2004, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2003, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2002, 1, 1, tzinfo=timezone.utc)},
    # {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2005, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2004, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2003, 1, 1, tzinfo=timezone.utc)},
    # {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2006, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2005, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2004, 1, 1, tzinfo=timezone.utc)},
    # {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2007, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2006, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2005, 1, 1, tzinfo=timezone.utc)},
    # {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2008, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2007, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2006, 1, 1, tzinfo=timezone.utc)},
    # {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2009, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2008, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2007, 1, 1, tzinfo=timezone.utc)},
    
    {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2010, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2009, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2008, 1, 1, tzinfo=timezone.utc)},
    {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2011, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2010, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2009, 1, 1, tzinfo=timezone.utc)},
    {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2012, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2011, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2010, 1, 1, tzinfo=timezone.utc)},
    {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2013, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2012, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2011, 1, 1, tzinfo=timezone.utc)},
    {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2014, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2013, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2012, 1, 1, tzinfo=timezone.utc)},
    {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2015, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2014, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2013, 1, 1, tzinfo=timezone.utc)},
    {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2016, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2015, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2014, 1, 1, tzinfo=timezone.utc)},
    {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2017, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2016, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2015, 1, 1, tzinfo=timezone.utc)},
    {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2018, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2017, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2016, 1, 1, tzinfo=timezone.utc)},
    {'symbols': Constants.Data.DJIA_2000_2004, 'end': datetime(2019, 1, 1, tzinfo=timezone.utc), 'last_val_date': datetime(2018, 1, 1, tzinfo=timezone.utc), 'last_train_date': datetime(2017, 7, 1, tzinfo=timezone.utc)},
]

In [ ]:
realized_returns = []
actions = []
for i in range(len(start_end_last_train_dates)):
    best_model = None
    best_loss = 100.
    episode_returns = None
    for j in range(1): 
        cur_actions, cur_returns, cur_model, cur_val_loss = evaluate_signal_predictor(
            start_end_last_train_dates[i]['symbols'],
            start_end_last_train_dates[i]['end'], 
            start_end_last_train_dates[i]['last_train_date'], 
            start_end_last_train_dates[i]['last_val_date'], 
            best_model)
        if cur_val_loss < best_loss: 
            logging.info('Found better val loss, updating episode_returns')
            best_loss = cur_val_loss
            episode_returns = cur_returns
            episode_actions = cur_actions
    logging.info(f'Len of episode_returns: {len(episode_returns)}')
    realized_returns.extend(episode_returns)
    actions.extend(episode_actions)
    
    # for j in range(1): 
    #     cur_rezlized_returns, model = evaluate_signal_predictor(
    #         start_end_last_train_dates[i]['start'], 
    #         start_end_last_train_dates[i]['last_val_date'], 
    #         start_end_last_train_dates[i]['last_train_date'])
    #     if np.mean(cur_rezlized_returns) > best_mean_return:
    #         best_mean_return = np.mean(cur_rezlized_returns)
    #         best_model = model

    # realized_returns.extend(
    #     evaluate_signal_predictor(
    #         start_end_last_train_dates[i]['end'], 
    #         start_end_last_train_dates[i]['last_train_date'], 
    #         start_end_last_train_dates[i]['last_val_date'], 
    #         best_model)[0])

Running experiment for 1970-01-02 00:00:00-05:00 to 2010-01-01 00:00:00+00:00 with train set last date 2008-01-01 00:00:00+00:00 and val set last date 2009-01-01 00:00:00+00:00


2025-12-08 16:38:00,806 - WARNING - No data found for ek_us_d.csv in root folder
2025-12-08 16:38:00,947 - WARNING - No data found for GM between 1970-01-02 00:00:00-05:00 and 2010-01-01 00:00:00+00:00
2025-12-08 16:38:03,222 - INFO - retrieved 27 assets
2025-12-08 16:38:03,223 - INFO - Using monolithic slices with -60 timestamps
2025-12-08 16:38:03,871 - INFO - Found 9589 train slices, 336 val slices, 336 test slices
2025-12-08 16:38:03,872 - INFO - Trained per-asset targets


(13759, 27, 60, 16) (13759, 27) (13759, 27) (13759, 27) (13759, 27) (366, 27, 60, 16) (366, 27) (366, 27) (366, 27) (366, 27)
Training model...


2025-12-08 16:38:17,466 - INFO - Model compiled with torch.compile()
2025-12-08 16:38:17,467 - INFO - Epoch 1/20
2025-12-08 16:38:19,672 - INFO - Train Loss: -0.0090        
2025-12-08 16:38:19,673 - INFO - Train Log_return: -0.0133
2025-12-08 16:38:19,673 - INFO - Train Mean_return: 0.0148
2025-12-08 16:38:19,673 - INFO - Val   Loss: 0.0504
2025-12-08 16:38:19,674 - INFO - Val   Log_return: 0.0371
2025-12-08 16:38:19,674 - INFO - Val   Mean_return: -0.0314
2025-12-08 16:38:19,674 - INFO - New best model found! Updating best state dict.
2025-12-08 16:38:19,676 - INFO - 
2025-12-08 16:38:19,676 - INFO - Epoch 2/20
2025-12-08 16:38:20,536 - INFO - Train Loss: -0.0131       
2025-12-08 16:38:20,537 - INFO - Train Log_return: -0.0211
2025-12-08 16:38:20,537 - INFO - Train Mean_return: 0.0239
2025-12-08 16:38:20,538 - INFO - Val   Loss: 0.0427
2025-12-08 16:38:20,539 - INFO - Val   Log_return: 0.0334
2025-12-08 16:38:20,539 - INFO - Val   Mean_return: -0.0294
2025-12-08 16:38:20,539 - INFO 

[PolicyGradient] [VAL] Epoch 0/10 — Loss: 0.0000


...

2025-12-08 16:40:29,166 - INFO - Model compiled with torch.compile()
2025-12-08 16:40:29,167 - INFO - Epoch 1/20
2025-12-08 16:40:31,116 - INFO - Train Loss: -0.0056        
2025-12-08 16:40:31,117 - INFO - Train Log_return: -0.0093
2025-12-08 16:40:31,117 - INFO - Train Mean_return: 0.0107
2025-12-08 16:40:31,117 - INFO - Val   Loss: -0.0040
2025-12-08 16:40:31,118 - INFO - Val   Log_return: -0.0053
2025-12-08 16:40:31,118 - INFO - Val   Mean_return: 0.0058
2025-12-08 16:40:31,118 - INFO - New best model found! Updating best state dict.
2025-12-08 16:40:31,120 - INFO - 
2025-12-08 16:40:31,120 - INFO - Epoch 2/20
2025-12-08 16:40:32,084 - INFO - Train Loss: -0.0064        
2025-12-08 16:40:32,085 - INFO - Train Log_return: -0.0102
2025-12-08 16:40:32,085 - INFO - Train Mean_return: 0.0115
2025-12-08 16:40:32,085 - INFO - Val   Loss: -0.0024
2025-12-08 16:40:32,086 - INFO - Val   Log_return: -0.0040
2025-12-08 16:40:32,086 - INFO - Val   Mean_return: 0.0046
2025-12-08 16:40:32,086 - IN

[PolicyGradient] [VAL] Epoch 0/10 — Loss: 0.0000


2025-12-08 16:40:41,920 - INFO - Found better val loss, updating episode_returns
2025-12-08 16:40:41,921 - INFO - Len of episode_returns: 13


Running experiment for 1970-01-02 00:00:00-05:00 to 2015-01-01 00:00:00+00:00 with train set last date 2013-01-01 00:00:00+00:00 and val set last date 2014-01-01 00:00:00+00:00


2025-12-08 16:40:43,011 - WARNING - No data found for ek_us_d.csv in root folder
2025-12-08 16:40:45,759 - INFO - retrieved 28 assets
2025-12-08 16:40:45,760 - INFO - Using monolithic slices with -60 timestamps
2025-12-08 16:40:46,683 - INFO - Found 10848 train slices, 333 val slices, 336 test slices
2025-12-08 16:40:46,684 - INFO - Trained per-asset targets


In [ ]:
len(realized_returns)

In [ ]:
plot_cumulative_wealth(
    returns_dict={
        'Signal Predictor': realized_returns,
    }, 
    start_time=config.data_config.val_set_last_date, 
    end_time=config.data_config.end,
    compare_to_baseline=False
)

In [ ]:
metrics = MetricsCalculator(
    metrics=DEFAULT_METRICS
)(realized_returns)
metrics

In [ ]:
plot_position_heatmap(actions[-20:], asset_names=sorted(list(Constants.Data.DJIA_2000_2004[:-2])))

In [ ]:
pd.Series(realized_returns).to_csv('realized_returns_ours.csv')

In [ ]:
[sum(action) for action in actions]